In [ ]:
import pathlib

# Access dataset using paths
frames = pathlib.Path("./frames")

train_frames = frames / "train"
train_norm_frames = train_frames / "norm"
train_weap_frames = train_frames / "weap"

test_frames = frames / "test"
test_norm_frames = test_frames / "norm"
test_weap_frames = test_frames / "weap"

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_generator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

print("Training data:")
training_data_generator = data_generator.flow_from_directory(
    train_frames, 
    batch_size=32, 
    class_mode='binary',
    subset='training'
)

print("Validation data:")
validation_data_generator = data_generator.flow_from_directory(
    train_frames,
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

In [ ]:
import urllib
from tensorflow.keras.applications.inception_v3 import InceptionV3

weights_url = "https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
weights_file = "inception_v3.h5"
urllib.request.urlretrieve(weights_url, weights_file)

# Instantiate the model
inception = InceptionV3(input_shape=(256, 256, 3), include_top=False, weights=None)

# load pre-trained weights
inception.load_weights(weights_file)

# freeze the layers
for layer in inception.layers:
    layer.trainable = False

inception.summary()

last_layer = inception.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import Model

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)

model = Model(inception.input, x)

In [ ]:
from tensorflow.keras.optimizers import RMSprop

# compile the model
model.compile(
    optimizer=RMSprop(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['acc']
)

# train the model (adjust the number of epochs from 1 to improve performance)
history = model.fit(
    training_data_generator,
    validation_data=validation_data_generator,
    epochs=5,
    verbose=1
)

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

# Visualize training results
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = np.array(range(5))

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
import tensorflow as tf

test_ds = tf.keras.utils.image_dataset_from_directory(test_frames)

model.evaluate(test_ds)